# DATA SPLIT - DEMO

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#project variables
demopath=os.getcwd()


## define a fixed samplerate

In [2]:
samplerate=200

## load dreams

In [3]:
signals, annotations, signalsMetadata = loadDREAMSSpindlesDemo(demopath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [4]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [5]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


In [6]:
signalsMetadata

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS
5,excerpt6.txt,CZ-A1,0006,1800,200,DREAMS
6,excerpt7.txt,CZ-A1,0007,1800,200,DREAMS
7,excerpt8.txt,CZ-A1,0008,1800,200,DREAMS


## create data split
LOOCV with 1 subjects for validation by fold

In [7]:
#define subjects used in the experiment, keep only annotated by two experts
usedSubjects=np.array(signalsMetadata['subjectId'])
usedSubjects=np.setdiff1d(usedSubjects,['0004','0007','0008'])
usedSubjects

array(['0001', '0002', '0003', '0005', '0006'], dtype=object)

In [8]:
valCount=1  #number of signals for validation (selected randomly)

trainSplits=[]
valSplits=[]
testSplits=[]
for i in range(len(usedSubjects)):
    thisTest=usedSubjects[i]
    thisNoTest=np.setdiff1d(usedSubjects,thisTest)
    thisVal=np.random.choice(thisNoTest,size=valCount,replace=False)
    thisTrain=np.setdiff1d(thisNoTest,thisVal)
    trainSplits.append(thisTrain)
    valSplits.append(thisVal)
    testSplits.append(thisTest)

dataSplits=pd.DataFrame({
    'train':trainSplits,
    'val':valSplits,
    'test':testSplits
})

dataSplits

,train,val,test
0,"[0002, 0003, 0006]",[0005],0001
1,"[0001, 0003, 0005]",[0006],0002
2,"[0001, 0002, 0006]",[0005],0003
3,"[0001, 0002, 0006]",[0003],0005
4,"[0002, 0003, 0005]",[0001],0006


In [9]:
dumpPickle("dataSplits_LOOCV_5S.pkl",dataSplits)